In [1]:
import pandas as pd
from functions import connectCollection, searchNear, searchNearWithoutLimit

df_filtered = pd.read_csv("../input/clean_df_companies.csv")
print("clean_df_companies.csv successfully imported")
# Import airports around
db, airports = connectCollection('companies', 'airports')
print("airports collection successfully loaded")

df_filtered["Closest_Airport"] = df_filtered.apply(lambda x: searchNear(
    x.longitude, x.latitude, airports, 40000), axis=1)
df_filtered = df_filtered[df_filtered['Closest_Airport'].astype(bool)]

# Import old companies around
db, oldcos = connectCollection('companies', 'oldCos')
print("oldCos collection successfully loaded")

df_filtered["Closest_oldCo"] = df_filtered.apply(lambda x: searchNear(
    x.longitude, x.latitude, oldcos, 300), axis=1)
df_filtered = df_filtered[df_filtered['Closest_oldCo'].astype(
    bool) == False]

# Import Starbucks around
db, starbucks = connectCollection('companies', 'starbucks')
print("starbucks collection successfully loaded")

df_filtered["Closest_Starbucks"] = df_filtered.apply(lambda x: searchNear(
    x.longitude, x.latitude, starbucks, 1000), axis=1)
df_filtered = df_filtered[df_filtered['Closest_Starbucks'].astype(
    bool)]

# Import Tech Companies with >$1m raised around
db, techCos = connectCollection('companies', 'techCos')
print("techCos collection successfully loaded")

df_filtered["Closest_techCo"] = df_filtered.apply(lambda x: searchNearWithoutLimit(
    x.longitude, x.latitude, techCos, 400), axis=1)
df_filtered = df_filtered[df_filtered['Closest_techCo'].astype(
    bool)]
df_filtered.reset_index(inplace=True, drop=True)
df_filtered.to_csv("../input/df_filtered")
print("df_filtered.csv successfully exported")

clean_df_companies.csv successfully imported
airports collection successfully loaded
oldCos collection successfully loaded
starbucks collection successfully loaded
techCos collection successfully loaded
df_filtered.csv successfully exported


In [8]:
df_filtered["Number_of_TechCo_around"] = df_filtered.Closest_techCo.apply(lambda x: len(x))

In [9]:
df_filtered

,Unnamed: 0,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,...,state_code,country_code,latitude,longitude,Tech/Other,Closest_Airport,Closest_oldCo,Closest_Starbucks,Closest_techCo,Number_of_TechCo_around
0,11,52cdef7c4bab8bd675297d99,Lala,lala,http://www.crunchbase.com/company/lala,http://www.lala.com,http://www.lala.com/#blog,http://www.lala.com/#blog,NaN,games_video,...,CA,USA,37.451151,-122.154369,Tech,"[{'_id': 5dd675f33b0eaf5b22d77dad, 'name': 'Sa...",[],"[{'_id': 5dd67f43e1d2f64cdaf41b93, 'city': 'Pa...","[{'_id': '52cdef7c4bab8bd675297d99', 'Unnamed:...",1
1,107,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,MI,USA,42.547807,-83.215498,Other,"[{'_id': 5dd675f33b0eaf5b22d77c91, 'name': 'De...",[],"[{'_id': 5dd67f43e1d2f64cdaf423e5, 'city': 'Bi...","[{'_id': '52cdef7c4bab8bd675297e67', 'Unnamed:...",1
2,108,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,CA,USA,33.658331,-117.861817,Other,"[{'_id': 5dd675f33b0eaf5b22d77dae, 'name': 'Jo...",[],"[{'_id': 5dd67f43e1d2f64cdaf4315c, 'city': 'Ne...","[{'_id': '52cdef7e4bab8bd67529bd27', 'Unnamed:...",1
3,117,52cdef7c4bab8bd675297dba,Google,google,http://www.crunchbase.com/company/google,http://google.com,http://googleblog.blogspot.com,http://googleblog.blogspot.com/feeds/posts/def...,google,search,...,DC,USA,38.901113,-77.027101,Other,"[{'_id': 5dd675f33b0eaf5b22d77bba, 'name': 'Wa...",[],"[{'_id': 5dd67f43e1d2f64cdaf41177, 'city': 'Wa...","[{'_id': '52cdef7d4bab8bd6752995f9', 'Unnamed:...",3
4,267,52cdef7c4bab8bd675297e47,ScanScout,scanscout,http://www.crunchbase.com/company/scanscout,http://www.scanscout.com,NaN,NaN,NaN,advertising,...,CA,USA,34.050887,-118.459995,Other,"[{'_id': 5dd675f33b0eaf5b22d77eb9, 'name': 'Lo...",[],"[{'_id': 5dd67f43e1d2f64cdaf428e3, 'city': 'Lo...","[{'_id': '52cdef7e4bab8bd67529a28d', 'Unnamed:...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,15632,52cdef7f4bab8bd67529c5e8,Shape Games,shape-games,http://www.crunchbase.com/company/shape-games,http://www.shapegame.ru,NaN,NaN,NaN,games_video,...,VA,USA,38.814208,-77.042423,Tech,"[{'_id': 5dd675f33b0eaf5b22d77bba, 'name': 'Wa...",[],"[{'_id': 5dd67f43e1d2f64cdaf4287c, 'city': 'Al...","[{'_id': '52cdef7f4bab8bd67529c5e8', 'Unnamed:...",2
457,15668,52cdef7f4bab8bd67529c60a,Technologies and Devices International,technologies-and-devices-international,http://www.crunchbase.com/company/technologies...,http://www.tdii.com,NaN,NaN,NaN,semiconductor,...,MD,USA,39.053372,-76.958117,Tech,"[{'_id': 5dd675f33b0eaf5b22d77bba, 'name': 'Wa...",[],"[{'_id': 5dd67f43e1d2f64cdaf4265a, 'city': 'Si...","[{'_id': '52cdef7f4bab8bd67529c60a', 'Unnamed:...",1
458,15677,52cdef7f4bab8bd67529c613,KooDooZ,koodooz,http://www.crunchbase.com/company/koodooz,http://www.koodooz.com,http://koodooz.blogspot.com,http://www.addthis.com/feed.php?pub=koodooz&h1...,KooDooZ,other,...,CA,USA,34.013266,-118.490269,Other,"[{'_id': 5dd675f33b0eaf5b22d77eb9, 'name': 'Lo...",[],"[{'_id': 5dd67f43e1d2f64cdaf45697, 'city': 'Sa...","[{'_id': '52cdef7f4bab8bd67529c613', 'Unnamed:...",4
459,15834,52cdef7f4bab8bd67529c6e4,Getyoo,getyoo,http://www.crunchbase.com/company/getyoo,http://www.getyoo.com,http://getyoo.tumblr.com/,http://getyoo.tumblr.com/rss,getyoo,mobile,...,NaN,BEL,50.839970,4.346472,Tech,"[{'_id': 5dd675f33b0eaf5b22d77d92, 'name': 'Br...",[],"[{'_id': 5dd67f43e1d2f64cdaf45ce5, 'city': 'Br...","[{'_id': '52cdef7f4bab8bd67529c6e4', 'Unnamed:...",1


In [23]:
df_check_techCos = df_filtered[["city", "Number_of_TechCo_around"]]
df_check_techCos = df_check_techCos.groupby("city").sum().sort_values(by="Number_of_TechCo_around", ascending=False)

In [26]:
df_check_techCos.head(25)

,Number_of_TechCo_around
city,
Atlanta,25
Chicago,25
Denver,25
Austin,22
San Mateo,21
London,17
Paris,16
Pittsburgh,16
New York,15


In [27]:
df_apartment = pd.read_csv("../input/apartment-rent-summary.csv")

In [33]:
df_apartment[df_apartment["Location"].str.contains("Atlanta|Chicago|Denver|Austin|San Mateo") == True].sort_values(by=["Price_3br"], ascending=False)

,Location,Location_Type,Year,Month,Price_Studio,Price_1br,Price_2br,Price_3br,Price_4br,Price_Change_MoM,Price_Change_YoY
608,"San Mateo, CA",City,2019,10,2875,3534,4439,5973,7103,-0.1,1.4
88,"Austin, TX",City,2019,10,983,1192,1471,1993,2398,0.4,3.2
225,"Denver, CO",City,2019,10,875,1069,1354,1967,2278,-0.6,0.6
175,"Chicago, IL",City,2019,10,940,1091,1283,1641,1953,0.0,1.2
80,"Atlanta, GA",City,2019,10,999,1047,1210,1588,1952,0.1,1.8
174,"Chicago Heights, IL",City,2019,10,798,926,1089,1393,1658,-0.1,0.8
495,"North Chicago, IL",City,2019,10,717,832,978,1251,1489,0.3,0.2


In [38]:

#http://austin.culturemap.com/news/innovation/07-03-19-austin-ranking-best-cities-startups-commercialcafe/

df_austin = df_filtered[df_filtered.city == "Austin"]
df_austin.reset_index(inplace=True, drop=True)
df_austin.to_csv("../input/df_austin.csv")

In [39]:
df_austin["Closest_Starbucks"][0]

[{'_id': ObjectId('5dd67f43e1d2f64cdaf44024'),
  'city': 'Austin',
  'name': '360 & Bee Caves',
  'country': 'US',
  'longitude': -97.8251037598,
  'latitude': 30.2924041748,
  'store_id': 116570,
  'geoJSON': {'type': 'Point',
   'coordinates': [-97.8251037598, 30.2924041748]}}]